In [2]:
df = pd.read_csv("input/train.csv", index_col=False)

In [3]:
df.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f282,f283,f284,f285,f286,f287,f288,f289,f290,target
0,1,2014-01-29,69,38.0,7.0,10.0,1.0,2001.0,2.0,11.0,...,1,0,0,2,8,1,0,19,2,9500000
1,2,2014-04-07,55,NaN,2.0,1.0,4.0,NaN,2.0,10.0,...,0,0,0,0,4,0,0,2,0,3837949
2,3,2012-05-18,30,16.0,2.0,NaN,NaN,NaN,NaN,NaN,...,46,9,2,11,38,1,8,97,11,6250000
3,4,2013-02-08,44,43.0,1.0,NaN,NaN,NaN,NaN,NaN,...,17,4,1,12,12,0,1,55,7,2000000
4,5,2014-01-10,45,28.0,3.0,5.0,2.0,1960.0,2.0,5.0,...,20,2,0,4,16,1,4,47,5,6700000


In [4]:
df.shape

(24376, 292)

# Preprocessing

## Check types

In [5]:
df.dtypes.value_counts()

int64      157
float64    119
object      16
dtype: int64

## Get notnums w/ indices

In [6]:
notnum = df.dtypes[df.dtypes == type(object)]
notnum

f1      object
f11     object
f12     object
f29     object
f33     object
f34     object
f35     object
f36     object
f37     object
f38     object
f39     object
f40     object
f106    object
f114    object
f118    object
f152    object
dtype: object

In [7]:
notnum_index = list(notnum.index)
print(notnum_index)

['f1', 'f11', 'f12', 'f29', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f106', 'f114', 'f118', 'f152']


In [8]:
df[notnum_index].head()

,f1,f11,f12,f29,f33,f34,f35,f36,f37,f38,f39,f40,f106,f114,f118,f152
0,2014-01-29,Investment,Juzhnoe Butovo,no,no,no,no,no,no,no,no,no,no,no,no,good
1,2014-04-07,OwnerOccupier,Poselenie Filimonkovskoe,no,no,no,no,no,no,no,no,no,no,no,no,no data
2,2012-05-18,Investment,Lomonosovskoe,no,no,no,no,no,no,no,no,no,no,no,no,satisfactory
3,2013-02-08,Investment,Juzhnoe Tushino,no,no,no,no,no,no,no,no,no,no,no,no,poor
4,2014-01-10,Investment,Ochakovo-Matveevskoe,no,yes,no,no,no,no,no,no,no,no,no,no,satisfactory


## Process f1

In [9]:
ymd = df['f1'].str.split('-', expand=True).astype('int64').rename(columns={0: 'f1y', 1: 'f1m', 2: 'f1d'})
ymd

,f1y,f1m,f1d
0,2014,1,29
1,2014,4,7
2,2012,5,18
3,2013,2,8
4,2014,1,10
...,...,...,...
24371,2014,12,21
24372,2012,5,10
24373,2013,12,18
24374,2014,12,8


In [10]:
df_f1 = df.drop(['f1'], axis=1)
df_f1.insert(1, 'f1y', ymd['f1y'])
df_f1.insert(2, 'f1m', ymd['f1m'])
df_f1.insert(3, 'f1d', ymd['f1d'])

In [11]:
notnum_index = ['f1y', 'f1m', 'f1d'] + notnum_index[1:]
print(notnum_index)

['f1y', 'f1m', 'f1d', 'f11', 'f12', 'f29', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f106', 'f114', 'f118', 'f152']


In [12]:
df_f1[notnum_index].head()

,f1y,f1m,f1d,f11,f12,f29,f33,f34,f35,f36,f37,f38,f39,f40,f106,f114,f118,f152
0,2014,1,29,Investment,Juzhnoe Butovo,no,no,no,no,no,no,no,no,no,no,no,no,good
1,2014,4,7,OwnerOccupier,Poselenie Filimonkovskoe,no,no,no,no,no,no,no,no,no,no,no,no,no data
2,2012,5,18,Investment,Lomonosovskoe,no,no,no,no,no,no,no,no,no,no,no,no,satisfactory
3,2013,2,8,Investment,Juzhnoe Tushino,no,no,no,no,no,no,no,no,no,no,no,no,poor
4,2014,1,10,Investment,Ochakovo-Matveevskoe,no,yes,no,no,no,no,no,no,no,no,no,no,satisfactory


## Process f11

In [13]:
f11_unique = df_f1['f11'].unique()
f11_unique

array(['Investment', 'OwnerOccupier'], dtype=object)

In [14]:
f11_encode = dict(zip(f11_unique, range(len(f11_unique))))
f11_encode

{'Investment': 0, 'OwnerOccupier': 1}

In [15]:
df_f11 = df_f1.replace({'f11': f11_encode})
df_f11[notnum_index].head()

,f1y,f1m,f1d,f11,f12,f29,f33,f34,f35,f36,f37,f38,f39,f40,f106,f114,f118,f152
0,2014,1,29,0,Juzhnoe Butovo,no,no,no,no,no,no,no,no,no,no,no,no,good
1,2014,4,7,1,Poselenie Filimonkovskoe,no,no,no,no,no,no,no,no,no,no,no,no,no data
2,2012,5,18,0,Lomonosovskoe,no,no,no,no,no,no,no,no,no,no,no,no,satisfactory
3,2013,2,8,0,Juzhnoe Tushino,no,no,no,no,no,no,no,no,no,no,no,no,poor
4,2014,1,10,0,Ochakovo-Matveevskoe,no,yes,no,no,no,no,no,no,no,no,no,no,satisfactory


## Process f12

In [16]:
f12_unique = df_f11['f12'].unique()
f12_unique

array(['Juzhnoe Butovo', 'Poselenie Filimonkovskoe', 'Lomonosovskoe',
       'Juzhnoe Tushino', 'Ochakovo-Matveevskoe', 'Poselenie Sosenskoe',
       'Orehovo-Borisovo Juzhnoe', 'Hovrino', 'Sokol',
       'Nagatinskij Zaton', "Kon'kovo", 'Obruchevskoe',
       'Poselenie Vnukovskoe', 'Preobrazhenskoe', 'Poselenie Moskovskij',
       'Bibirevo', 'Jasenevo', 'Mitino', 'Birjulevo Vostochnoe',
       'Koptevo', 'Beskudnikovskoe', 'Butyrskoe', 'Losinoostrovskoe',
       'Novo-Peredelkino', 'Caricyno', 'Veshnjaki', 'Tverskoe', 'Perovo',
       'Jaroslavskoe', 'Strogino', 'Nekrasovka', 'Horoshevo-Mnevniki',
       'Filevskij Park', 'Bogorodskoe', 'Savelki',
       'Poselenie Desjonovskoe', "Chertanovo Central'noe",
       'Timirjazevskoe', 'Zapadnoe Degunino', 'Pokrovskoe Streshnevo',
       'Severnoe Butovo', 'Kapotnja', 'Sviblovo', 'Danilovskoe',
       'Sokolinaja Gora', 'Vostochnoe Izmajlovo', 'Matushkino',
       'Krjukovo', "Krasnosel'skoe", 'Ajeroport', 'Taganskoe',
       'Cheremushki

In [17]:
f12_encode = dict(zip(f12_unique, range(len(f12_unique))))
f12_encode

{'Juzhnoe Butovo': 0,
 'Poselenie Filimonkovskoe': 1,
 'Lomonosovskoe': 2,
 'Juzhnoe Tushino': 3,
 'Ochakovo-Matveevskoe': 4,
 'Poselenie Sosenskoe': 5,
 'Orehovo-Borisovo Juzhnoe': 6,
 'Hovrino': 7,
 'Sokol': 8,
 'Nagatinskij Zaton': 9,
 "Kon'kovo": 10,
 'Obruchevskoe': 11,
 'Poselenie Vnukovskoe': 12,
 'Preobrazhenskoe': 13,
 'Poselenie Moskovskij': 14,
 'Bibirevo': 15,
 'Jasenevo': 16,
 'Mitino': 17,
 'Birjulevo Vostochnoe': 18,
 'Koptevo': 19,
 'Beskudnikovskoe': 20,
 'Butyrskoe': 21,
 'Losinoostrovskoe': 22,
 'Novo-Peredelkino': 23,
 'Caricyno': 24,
 'Veshnjaki': 25,
 'Tverskoe': 26,
 'Perovo': 27,
 'Jaroslavskoe': 28,
 'Strogino': 29,
 'Nekrasovka': 30,
 'Horoshevo-Mnevniki': 31,
 'Filevskij Park': 32,
 'Bogorodskoe': 33,
 'Savelki': 34,
 'Poselenie Desjonovskoe': 35,
 "Chertanovo Central'noe": 36,
 'Timirjazevskoe': 37,
 'Zapadnoe Degunino': 38,
 'Pokrovskoe Streshnevo': 39,
 'Severnoe Butovo': 40,
 'Kapotnja': 41,
 'Sviblovo': 42,
 'Danilovskoe': 43,
 'Sokolinaja Gora': 44,
 'V

In [18]:
df_f12 = df_f11.replace({'f12': f12_encode})
df_f12[notnum_index].head()

,f1y,f1m,f1d,f11,f12,f29,f33,f34,f35,f36,f37,f38,f39,f40,f106,f114,f118,f152
0,2014,1,29,0,0,no,no,no,no,no,no,no,no,no,no,no,no,good
1,2014,4,7,1,1,no,no,no,no,no,no,no,no,no,no,no,no,no data
2,2012,5,18,0,2,no,no,no,no,no,no,no,no,no,no,no,no,satisfactory
3,2013,2,8,0,3,no,no,no,no,no,no,no,no,no,no,no,no,poor
4,2014,1,10,0,4,no,yes,no,no,no,no,no,no,no,no,no,no,satisfactory


## Process f29 - f118

In [19]:
notnum_bools = notnum_index[5:-1]
print(notnum_bools)

['f29', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f106', 'f114', 'f118']


In [20]:
bools_unique = set(np.array([df_f12[col].unique() for col in notnum_bools]).flatten())
bools_unique

{'no', 'yes'}

In [21]:
bools_encode = dict(zip(bools_unique, range(len(bools_unique))))
bools_encode

{'yes': 0, 'no': 1}

In [22]:
df_bools = df_f12.replace(dict.fromkeys(notnum_bools, bools_encode))
df_bools[notnum_index].head()

,f1y,f1m,f1d,f11,f12,f29,f33,f34,f35,f36,f37,f38,f39,f40,f106,f114,f118,f152
0,2014,1,29,0,0,1,1,1,1,1,1,1,1,1,1,1,1,good
1,2014,4,7,1,1,1,1,1,1,1,1,1,1,1,1,1,1,no data
2,2012,5,18,0,2,1,1,1,1,1,1,1,1,1,1,1,1,satisfactory
3,2013,2,8,0,3,1,1,1,1,1,1,1,1,1,1,1,1,poor
4,2014,1,10,0,4,1,0,1,1,1,1,1,1,1,1,1,1,satisfactory


## Process f152

In [23]:
f152_unique = df_bools['f152'].unique()
f152_unique

array(['good', 'no data', 'satisfactory', 'poor', 'excellent'],
      dtype=object)

In [24]:
f152_encode = dict(zip(['no data', 'poor', 'satisfactory', 'good', 'excellent'], range(5)))
f152_encode

{'no data': 0, 'poor': 1, 'satisfactory': 2, 'good': 3, 'excellent': 4}

In [25]:
df_f152 = df_bools.replace({'f152': f152_encode})
df_f152[notnum_index].head()

,f1y,f1m,f1d,f11,f12,f29,f33,f34,f35,f36,f37,f38,f39,f40,f106,f114,f118,f152
0,2014,1,29,0,0,1,1,1,1,1,1,1,1,1,1,1,1,3
1,2014,4,7,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0
2,2012,5,18,0,2,1,1,1,1,1,1,1,1,1,1,1,1,2
3,2013,2,8,0,3,1,1,1,1,1,1,1,1,1,1,1,1,1
4,2014,1,10,0,4,1,0,1,1,1,1,1,1,1,1,1,1,2


## Obtain result

In [26]:
df_final = df_f152.copy()
df_final.head()

,id,f1y,f1m,f1d,f2,f3,f4,f5,f6,f7,...,f282,f283,f284,f285,f286,f287,f288,f289,f290,target
0,1,2014,1,29,69,38.0,7.0,10.0,1.0,2001.0,...,1,0,0,2,8,1,0,19,2,9500000
1,2,2014,4,7,55,NaN,2.0,1.0,4.0,NaN,...,0,0,0,0,4,0,0,2,0,3837949
2,3,2012,5,18,30,16.0,2.0,NaN,NaN,NaN,...,46,9,2,11,38,1,8,97,11,6250000
3,4,2013,2,8,44,43.0,1.0,NaN,NaN,NaN,...,17,4,1,12,12,0,1,55,7,2000000
4,5,2014,1,10,45,28.0,3.0,5.0,2.0,1960.0,...,20,2,0,4,16,1,4,47,5,6700000


## Check NaNs

In [27]:
nans_cols = df_final.isna().sum()
nans_cols[nans_cols > 0].count()

51

In [28]:
df_final[nans_cols[nans_cols > 0].index].dtypes.unique()

array([dtype('float64')], dtype=object)

In [29]:
nans_rows = df_final.isna().sum(axis=1)
nans_rows[nans_rows >= 40]

172      40
554      40
785      40
1052     40
1359     40
         ..
22921    40
23466    40
23550    40
24236    40
24316    41
Length: 155, dtype: int64

In [30]:
print(list(nans_rows[nans_rows >= 40].index))

[172, 554, 785, 1052, 1359, 1495, 1646, 1812, 1997, 2133, 2249, 2302, 2529, 2542, 2678, 2684, 2781, 2801, 2861, 3251, 3254, 3699, 3904, 3959, 4208, 4365, 4544, 4656, 4834, 4929, 4941, 5151, 5159, 5277, 5474, 5621, 5634, 5691, 5805, 6092, 6191, 6256, 6347, 6388, 6524, 6862, 6876, 7066, 7202, 7259, 7506, 7508, 7575, 7618, 7625, 7694, 7796, 7980, 8119, 8184, 8484, 8673, 8834, 9290, 9632, 9693, 9786, 10247, 10427, 10572, 10639, 11016, 11281, 11394, 11648, 11857, 12249, 12556, 12625, 13322, 13330, 13498, 13592, 13696, 13772, 13911, 13996, 14014, 14062, 14331, 14345, 14518, 14925, 14999, 15026, 15086, 15313, 15359, 15493, 15679, 15812, 15916, 16036, 16129, 16532, 16820, 16965, 17116, 17534, 17574, 17971, 18011, 18037, 18053, 18256, 18311, 18334, 18429, 18475, 18532, 18630, 18632, 18691, 19155, 19187, 19253, 19429, 19523, 19792, 20051, 20087, 20279, 20435, 20650, 20927, 21346, 21410, 21508, 21570, 21735, 22029, 22096, 22195, 22210, 22235, 22258, 22324, 22356, 22477, 22608, 22921, 23466, 23550

## Remove too dirty entries

In [31]:
df_nans = df_final.drop(list(nans_rows[nans_rows >= 40].index))
df_nans.shape

(24221, 294)

## Impute missing values with KNN

In [32]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(weights='distance')

In [33]:
df_imp = imputer.fit_transform(df_nans)

In [34]:
df_imp2 = pd.DataFrame(df_imp, columns=df_final.columns)
df_imp2.head()

,id,f1y,f1m,f1d,f2,f3,f4,f5,f6,f7,...,f282,f283,f284,f285,f286,f287,f288,f289,f290,target
0,1.0,2014.0,1.0,29.0,69.0,38.00000,7.0,10.00000,1.000000,2001.000000,...,1.0,0.0,0.0,2.0,8.0,1.0,0.0,19.0,2.0,9500000.0
1,2.0,2014.0,4.0,7.0,55.0,48.13174,2.0,1.00000,4.000000,1591.150833,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2.0,0.0,3837949.0
2,3.0,2012.0,5.0,18.0,30.0,16.00000,2.0,7.29370,3.732457,1961.927529,...,46.0,9.0,2.0,11.0,38.0,1.0,8.0,97.0,11.0,6250000.0
3,4.0,2013.0,2.0,8.0,44.0,43.00000,1.0,8.78533,1.428214,1970.661667,...,17.0,4.0,1.0,12.0,12.0,0.0,1.0,55.0,7.0,2000000.0
4,5.0,2014.0,1.0,10.0,45.0,28.00000,3.0,5.00000,2.000000,1960.000000,...,20.0,2.0,0.0,4.0,16.0,1.0,4.0,47.0,5.0,6700000.0


In [35]:
imputed = nans_cols[nans_cols > 0]
df_full = df_nans.copy()
df_full.reset_index(drop=True, inplace=True)
df_full.head()

,id,f1y,f1m,f1d,f2,f3,f4,f5,f6,f7,...,f282,f283,f284,f285,f286,f287,f288,f289,f290,target
0,1,2014,1,29,69,38.0,7.0,10.0,1.0,2001.0,...,1,0,0,2,8,1,0,19,2,9500000
1,2,2014,4,7,55,NaN,2.0,1.0,4.0,NaN,...,0,0,0,0,4,0,0,2,0,3837949
2,3,2012,5,18,30,16.0,2.0,NaN,NaN,NaN,...,46,9,2,11,38,1,8,97,11,6250000
3,4,2013,2,8,44,43.0,1.0,NaN,NaN,NaN,...,17,4,1,12,12,0,1,55,7,2000000
4,5,2014,1,10,45,28.0,3.0,5.0,2.0,1960.0,...,20,2,0,4,16,1,4,47,5,6700000


In [36]:
df_full[-3:]

,id,f1y,f1m,f1d,f2,f3,f4,f5,f6,f7,...,f282,f283,f284,f285,f286,f287,f288,f289,f290,target
24218,24374,2013,12,18,85,NaN,16.0,17.0,1.0,1.0,...,3,1,0,3,9,0,1,10,2,9609180
24219,24375,2014,12,8,47,27.0,2.0,10.0,2.0,1958.0,...,217,86,11,118,176,1,64,146,13,12900000
24220,24376,2015,4,30,85,60.0,10.0,22.0,4.0,2015.0,...,1,1,0,2,12,0,0,9,0,10371441


In [37]:
df_full[imputed.index] = df_imp2[imputed.index]
df_full.head()

,id,f1y,f1m,f1d,f2,f3,f4,f5,f6,f7,...,f282,f283,f284,f285,f286,f287,f288,f289,f290,target
0,1,2014,1,29,69,38.00000,7.0,10.00000,1.000000,2001.000000,...,1,0,0,2,8,1,0,19,2,9500000
1,2,2014,4,7,55,48.13174,2.0,1.00000,4.000000,1591.150833,...,0,0,0,0,4,0,0,2,0,3837949
2,3,2012,5,18,30,16.00000,2.0,7.29370,3.732457,1961.927529,...,46,9,2,11,38,1,8,97,11,6250000
3,4,2013,2,8,44,43.00000,1.0,8.78533,1.428214,1970.661667,...,17,4,1,12,12,0,1,55,7,2000000
4,5,2014,1,10,45,28.00000,3.0,5.00000,2.000000,1960.000000,...,20,2,0,4,16,1,4,47,5,6700000


In [38]:
True in df_full.isna().values

False

# Imports

In [1]:
import pandas as pd
import numpy as np

In [2]:
np.random.seed(3927)

In [32]:
nans = df_train.dtypes[df_train.dtypes == 'float64'].index
df_train[nans]
# ints_nans[ints_nans > 0]

,f3,f4,f5,f6,f7,f8,f9,f10,f13,f15,...,f245,f246,f252,f253,f254,f268,f269,f275,f276,f277
0,38.0,7.0,10.0,1.0,2001.0,2.0,11.0,2.0,2.615514e+07,0.137846,...,34.32,2.85,776.92,1230.77,1003.85,28.67,4.94,725.00,1175.00,950.00
1,NaN,2.0,1.0,4.0,NaN,2.0,10.0,NaN,3.574795e+07,0.548992,...,53.08,3.55,NaN,NaN,NaN,47.85,3.54,600.00,1000.00,800.00
2,16.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,3.326419e+06,0.051881,...,12.27,4.00,764.97,1271.57,1018.27,15.80,5.81,776.46,1293.67,1035.06
3,43.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,7.887684e+06,0.222646,...,17.77,13.01,688.24,1169.12,928.68,21.71,8.68,722.99,1211.23,967.11
4,28.0,3.0,5.0,2.0,1960.0,2.0,5.0,NaN,1.752651e+07,0.133373,...,18.50,16.62,811.90,1369.05,1090.48,23.17,11.27,752.08,1260.42,1006.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24371,NaN,7.0,17.0,1.0,NaN,2.0,1.0,1.0,2.553630e+07,0.496315,...,44.85,1.97,680.00,1200.00,940.00,35.53,5.77,758.82,1294.12,1026.47
24372,28.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,7.887684e+06,0.222646,...,15.90,13.30,876.74,1465.12,1170.93,21.64,6.92,804.55,1344.16,1074.35
24373,NaN,16.0,17.0,1.0,1.0,3.0,1.0,NaN,8.591335e+06,0.058543,...,31.34,5.79,827.27,1409.09,1118.18,31.26,6.06,836.36,1409.09,1122.73
24374,27.0,2.0,10.0,2.0,1958.0,2.0,6.0,4.0,1.249544e+07,0.027651,...,8.01,9.04,836.48,1383.20,1109.84,8.54,13.90,875.49,1441.24,1158.36


In [99]:
nan_columns = df_train.dtypes[df_train.dtypes == 'float64'].index
nan_is_int = df_train[nan_columns].replace({np.nan: 0}).applymap(lambda x: x.is_integer()).all()
groups = nan_is_int.groupby(nan_is_int).groups
# nan_int = nan_is_int[nan_is_int == True].index  # Get true integers
# nan_float = nan_is_int[nan_is_int == False].index  # Get true floats

TypeError: unhashable type: 'list'

In [78]:
nan_int

Index(['f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f18', 'f21', 'f24',
       'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77',
       'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f99'],
      dtype='object')

In [79]:
nan_float

Index(['f13', 'f15', 'f16', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91',
       'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f100', 'f101', 'f103',
       'f104', 'f105', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f115',
       'f117', 'f119', 'f121', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128',
       'f129', 'f130', 'f131', 'f132', 'f133', 'f134', 'f135', 'f136', 'f137',
       'f138', 'f139', 'f140', 'f141', 'f142', 'f143', 'f144', 'f145', 'f146',
       'f147', 'f148', 'f149', 'f150', 'f151', 'f153', 'f154', 'f160', 'f161',
       'f162', 'f176', 'f177', 'f183', 'f184', 'f185', 'f199', 'f200', 'f206',
       'f207', 'f208', 'f222', 'f223', 'f229', 'f230', 'f231', 'f245', 'f246',
       'f252', 'f253', 'f254', 'f268', 'f269', 'f275', 'f276', 'f277'],
      dtype='object')

In [87]:
nan_is_int.groupby(nan_is_int).groups[True]

Index(['f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f18', 'f21', 'f24',
       'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77',
       'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f99'],
      dtype='object')

In [67]:
nans_ints = df_train[nans].replace({np.nan: 0}).applymap(lambda x: x.is_integer()).all()
for k, v1, v2 in zip(nans_ints.index, nans_ints, df_train[nans_ints.index].isna().sum()):
    print(k, v1, v2)

f3 True 5111
f4 True 134
f5 True 7650
f6 True 7650
f7 True 10885
f8 True 7650
f9 True 7650
f10 True 10842
f13 False 0
f15 False 0
f16 False 0
f18 True 5331
f21 True 5329
f24 True 11486
f68 True 4009
f69 True 4009
f70 True 4009
f71 True 4009
f72 True 4009
f73 True 4009
f74 True 4009
f75 True 4009
f76 True 4009
f77 True 4009
f78 True 4009
f79 True 4009
f80 True 4009
f81 True 4009
f82 True 4009
f83 True 4009
f85 False 0
f86 False 0
f87 False 22
f88 False 22
f89 False 0
f90 False 0
f91 False 0
f92 False 0
f93 False 0
f94 False 0
f95 False 0
f96 False 0
f97 False 22
f98 False 22
f99 True 22
f100 False 0
f101 False 0
f103 False 0
f104 False 0
f105 False 0
f107 False 0
f108 False 0
f109 False 0
f110 False 0
f111 False 0
f112 False 0
f115 False 0
f117 False 0
f119 False 0
f121 False 0
f123 False 0
f124 False 0
f125 False 0
f126 False 0
f127 False 0
f128 False 0
f129 False 0
f130 False 0
f131 False 0
f132 False 0
f133 False 0
f134 False 0
f135 False 0
f136 False 0
f137 False 0
f138 False 0
f139

In [ ]:
float = df_train.dtypes[df_train.dtypes == 'int64'].index

In [31]:
for col in df_train.dtypes[df_train.dtypes == 'float64'].index:
    print(f"{col}: {df_train[col][df_train[col].isna() == False].apply(lambda x: x.is_integer()).all()}")

f3: True
f4: True
f5: True
f6: True
f7: True
f8: True
f9: True
f10: True
f13: False
f15: False
f16: False
f18: True
f21: True
f24: True
f68: True
f69: True
f70: True
f71: True
f72: True
f73: True
f74: True
f75: True
f76: True
f77: True
f78: True
f79: True
f80: True
f81: True
f82: True
f83: True
f85: False
f86: False
f87: False
f88: False
f89: False
f90: False
f91: False
f92: False
f93: False
f94: False
f95: False
f96: False
f97: False
f98: False
f99: True
f100: False
f101: False
f103: False
f104: False
f105: False
f107: False
f108: False
f109: False
f110: False
f111: False
f112: False
f115: False
f117: False
f119: False
f121: False
f123: False
f124: False
f125: False
f126: False
f127: False
f128: False
f129: False
f130: False
f131: False
f132: False
f133: False
f134: False
f135: False
f136: False
f137: False
f138: False
f139: False
f140: False
f141: False
f142: False
f143: False
f144: False
f145: False
f146: False
f147: False
f148: False
f149: False
f150: False
f151: False
f153: False


In [19]:
def func1(s1, s2):
    return s1 + ' is', s2 + ' is not'

def func2(s1, s2):
    print(s1 + ' cool')
    print(s2 + ' weird')

func2(*func1('C#', 'Python'))

C# is cool
Python is not weird


# Preprocess

In [3]:
def process_f1(df_train, df_test, index):
    ymd_train = df_train['f1'].str.split('-', expand=True).astype('int64').rename(columns={0: 'year', 1: 'month', 2: 'day'})
    ymd_test = df_test['f1'].str.split('-', expand=True).astype('int64').rename(columns={0: 'year', 1: 'month', 2: 'day'})
    return pd.concat([df_train, ymd_train], axis=1).drop(['f1'], axis=1), \
           pd.concat([df_test, ymd_test], axis=1).drop(['f1'], axis=1), \
           index[1:]

In [4]:
def process_f11(df_train, df_test, index):
    f11_unique = df_train['f11'].unique()
    f11_encode = dict(zip(f11_unique, np.eye(len(f11_unique))))
    encode_train = pd.DataFrame([np.nan if k == np.nan else f11_encode[k] for k in df_train['f11']], columns=f11_unique)
    encode_test = pd.DataFrame([np.nan if k == np.nan else f11_encode[k] for k in df_test['f11']], columns=f11_unique)
    return pd.concat([df_train, encode_train], axis=1).drop(['f11'], axis=1), \
           pd.concat([df_test, encode_test], axis=1).drop(['f11'], axis=1), \
           index[1:]

In [5]:
def process_f12(df_train, df_test, index):
    f12_unique = df_train['f12'].unique()
    f12_encode = dict(zip(f12_unique, np.eye(len(f12_unique))))
    encode_train = pd.DataFrame([np.nan if k == np.nan else f12_encode[k] for k in df_train['f12']], columns=f12_unique)
    encode_test = pd.DataFrame([np.nan if k == np.nan else f12_encode[k] for k in df_test['f12']], columns=f12_unique)
    return pd.concat([df_train, encode_train], axis=1).drop(['f12'], axis=1), \
           pd.concat([df_test, encode_test], axis=1).drop(['f12'], axis=1), \
           index[1:]

In [6]:
def process_bool(df_train, df_test, index):
    bool_index = index[:-1]
    bool_encode = {'no': 0, 'yes': 1}
    encoding = dict.fromkeys(bool_index, bool_encode)
    return df_train.replace(encoding), \
           df_test.replace(encoding), \
           [index[-1]]

In [7]:
def process_f152(df_train, df_test, index):
    f152_encode = dict(zip(['no data', 'poor', 'satisfactory', 'good', 'excellent'], range(5)))
    encoding = {'f152': f152_encode}
    return df_train.replace(encoding), \
           df_test.replace(encoding), \
           index

In [8]:
def process_categorical(df_train, df_test):
    cat = df_train.dtypes[df_train.dtypes == type(object)]
    cat_index = list(cat.index)
    
    df_train_res, df_test_res, cat_index = process_f1(df_train, df_test, cat_index)
    df_train_res, df_test_res, cat_index = process_f11(df_train_res, df_test_res, cat_index)
    df_train_res, df_test_res, cat_index = process_f12(df_train_res, df_test_res, cat_index)
    df_train_res, df_test_res, cat_index = process_bool(df_train_res, df_test_res, cat_index)
    df_train_res, df_test_res, cat_index = process_f152(df_train_res, df_test_res, cat_index)
    return df_train_res, df_test_res

In [9]:
from sklearn.impute import SimpleImputer, KNNImputer

def impute(df):
    # Setup imputers
    imputer_float = KNNImputer(n_neighbors=10, weights='distance')
    imputer_int = SimpleImputer(strategy='median')

    # Find columns, containing NaN, and split into int/float
    nan_columns = df.columns[df.isna().any()].to_list()
    nan_is_int= df[nan_columns].replace({np.nan: 0}).applymap(lambda x: x.is_integer()).all()
    nan_groups = nan_is_int.groupby(nan_is_int).groups  # Groups of "true" integers and floats
    df_floats, df_ints = df[nan_groups[False]], df[nan_groups[True]]

    # Impute missing values
    imp_float = pd.DataFrame(imputer_float.fit_transform(df_floats), columns=df_floats.columns)
    imp_int = pd.DataFrame(imputer_int.fit_transform(df_ints), columns=df_ints.columns)

    return pd.concat([df.drop(nan_columns, axis=1), imp_float, imp_int], axis=1)

def impute_na(df_train, df_test):
    return impute(df_train), impute(df_test)

In [60]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer

def transform(df_train, df_test):
    transformer = Normalizer()
    return pd.DataFrame(transformer.fit_transform(df_train), columns=df_train.columns), \
           pd.DataFrame(transformer.fit_transform(df_test), columns=df_test.columns)

In [11]:
def preprocess(df_train, df_test):
    df_train_res, df_test_res = process_categorical(df_train, df_test)
    df_train_res, df_test_res = impute_na(df_train_res, df_test_res)
    df_train_res, df_test_res = transform(df_train_res, df_test_res)
    return df_train_res, df_test_res

In [12]:
def prepare_dataset(df_train, df_test):
    return df_train.drop(['id', 'target'], axis=1), \
           df_train['target'], \
           df_test.drop('id', axis=1)

In [23]:
def load(train_path, test_path):
    df_train = pd.read_csv(train_path, index_col=False)
    df_test = pd.read_csv(test_path, index_col=False)
    X_train, y_train, X_test = prepare_dataset(df_train, df_test)
    X_train, X_test = preprocess(X_train, X_test)
    return X_train, y_train, X_test

In [61]:
X_train, y_train, X_test = load("input/train.csv", "input/test.csv")

In [62]:
X_train.head()

,f2,f13,f14,f15,f16,f17,f19,f20,f22,f23,...,f75,f76,f77,f78,f79,f80,f81,f82,f83,f99
0,0.000003,0.999836,0.006815,5.269468e-09,1.571761e-09,0.000538,4.204987e-07,0.000569,4.969530e-07,3.822715e-08,...,3.822715e-08,0.000003,1.911358e-07,0.000064,1.299723e-06,1.142992e-05,0.000017,0.000004,3.054350e-05,1.796676e-06
1,0.000002,0.999989,0.000071,1.535712e-08,9.694461e-10,0.000005,0.000000e+00,0.000005,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000,0.000000e+00,0.000008,0.000000e+00,5.594666e-08,0.000004,0.000002,6.713599e-07,6.713599e-07
2,0.000007,0.776825,0.019580,1.211579e-08,0.000000e+00,0.001095,1.167660e-06,0.001174,1.634724e-06,2.335320e-07,...,0.000000e+00,0.000000,0.000000e+00,0.000049,0.000000e+00,0.000000e+00,0.000036,0.000008,5.604768e-06,9.808343e-06
3,0.000005,0.960755,0.012721,2.711932e-08,2.299471e-08,0.000729,3.654134e-07,0.000748,4.872179e-07,0.000000e+00,...,0.000000e+00,0.000000,0.000000e+00,0.000040,3.654134e-07,4.872179e-07,0.000026,0.000011,2.436090e-06,4.872179e-07
4,0.000003,0.997675,0.006765,7.592085e-09,1.761673e-08,0.000424,1.138476e-07,0.000439,1.707714e-07,0.000000e+00,...,0.000000e+00,0.000000,0.000000e+00,0.000017,0.000000e+00,1.138476e-07,0.000008,0.000006,3.244656e-06,1.878485e-06


In [63]:
True in X_train.isna().values

False

In [64]:
y_train.head()

0    9500000
1    3837949
2    6250000
3    2000000
4    6700000
Name: target, dtype: int64

In [65]:
X_test.head()

,f2,f13,f14,f15,f16,f17,f19,f20,f22,f23,...,f75,f76,f77,f78,f79,f80,f81,f82,f83,f99
0,0.000002,0.999957,0.000157,1.943485e-08,2.788976e-10,0.000011,0.000000e+00,0.000010,0.000000e+00,0.000000e+00,...,0.0,0.000000e+00,0.000000e+00,0.000011,0.000000,7.831653e-08,0.000005,0.000003,9.397984e-07,9.397984e-07
1,0.000003,0.999973,0.000157,1.943516e-08,2.789021e-10,0.000011,0.000000e+00,0.000010,0.000000e+00,0.000000e+00,...,0.0,0.000000e+00,0.000000e+00,0.000011,0.000000,7.831780e-08,0.000005,0.000003,9.398136e-07,9.398136e-07
2,0.000003,0.511800,0.005279,4.583622e-09,5.473705e-12,0.000297,2.801540e-07,0.000448,2.801540e-07,7.003851e-08,...,0.0,3.501925e-07,7.003851e-08,0.000046,0.000018,7.354043e-06,0.000011,0.000005,3.992195e-06,3.501925e-07
3,0.000002,0.995239,0.009234,2.009066e-08,2.443350e-09,0.000550,4.753063e-07,0.000565,5.347196e-07,0.000000e+00,...,0.0,6.535462e-07,0.000000e+00,0.000027,0.000003,2.495358e-06,0.000006,0.000010,5.347196e-06,1.544746e-06
4,0.000025,0.926716,0.012046,5.140045e-08,0.000000e+00,0.000632,9.867259e-07,0.000666,9.867259e-07,0.000000e+00,...,0.0,0.000000e+00,0.000000e+00,0.000043,0.000000,0.000000e+00,0.000028,0.000004,1.065664e-05,6.512391e-06


In [66]:
True in X_test.isna().values

False

In [67]:
X_train.shape, y_train.shape, X_test.shape

((24376, 438), (24376,), (6095, 438))

# Train pipeline

In [69]:
from sklearn.metrics import make_scorer

def rmsle(y, y_pred, **kwargs):
    # Allow negative predictions 
    # by incorporating prediction sign
    terms = [np.sign(pred) * np.log(1 + np.sign(pred) * pred) - np.log(1 + targ) for pred, targ in zip(y_pred, y)]
    return np.sqrt(np.sum(np.power(terms, 2.0)) / len(y))

rmsle_scorer = make_scorer(rmsle, greater_is_better=False)

In [70]:
from sklearn.model_selection import cross_val_score, RandomizedSearchCV

def train(model, X, y, score=True, cv=5, tune=False, params=None, n_iter=10):
    if tune:
        if score:
            print(f"Pre-tune CV scores: {cross_val_score(model, X, y, cv=cv, scoring=rmsle_scorer)}")
        
        rscv = RandomizedSearchCV(model, params, n_iter=n_iter, scoring=rmsle_scorer, refit=True, cv=cv, random_state=3927)
        rscv.fit(X, y)
        print(f"Best params: {rscv.best_params_}")
        model_tr = rscv.best_estimator_

        if score:
            print(f"Post-tune CV scores: {cross_val_score(model_tr, X, y, cv=cv, scoring=rmsle_scorer)}")
    else:
        model_tr = model.fit(X, y)
    
        if score:
            print(f"CV scores: {cross_val_score(model_tr, X, y, cv=cv, scoring=rmsle_scorer)}")

    return model_tr

# Test pipeline

In [71]:
def test(model, X):
    pred = model.predict(X)
    data = list(zip(range(1, len(pred) + 1), pred))
    return pd.DataFrame(data, columns=['id', 'prediction'])

# Setup model

## Ridge

In [72]:
from sklearn.linear_model import Ridge
from scipy.stats import loguniform, uniform

In [73]:
model = Ridge(random_state=3927)
params = {'alpha': loguniform(1, 10000)}

model = train(model, X_train, y_train, tune=True, params=params, n_iter=100)
model

Pre-tune CV scores: [-0.60000799 -0.60348582 -0.60745596 -0.60441781 -0.62379452]
Best params: {'alpha': 1.738157608461319}
Post-tune CV scores: [-0.60021189 -0.6031775  -0.60753366 -0.60417195 -0.62365405]


Ridge(alpha=1.738157608461319, random_state=3927)

In [74]:
model = Ridge(random_state=3927)
params = {'alpha': loguniform(0.001, 1)}

model = train(model, X_train, y_train, tune=True, params=params, n_iter=100)
model

Pre-tune CV scores: [-0.60000799 -0.60348582 -0.60745596 -0.60441781 -0.62379452]
Best params: {'alpha': 0.9653009578533324}
Post-tune CV scores: [-0.59999686 -0.60353432 -0.60745569 -0.60444701 -0.62382557]


Ridge(alpha=0.9653009578533324, random_state=3927)

## ElasticNet

In [75]:
from sklearn.linear_model import ElasticNet
from scipy.stats import loguniform, uniform

In [76]:
model = ElasticNet(max_iter=500, selection='random', random_state=3927)
params = {'alpha': loguniform(0.01, 100), 'l1_ratio': uniform(0.1, 0.8)}

model = train(model, X_train, y_train, tune=True, params=params, n_iter=10)
model

Pre-tune CV scores: [-0.61793116 -0.61371143 -0.62756285 -0.62152099 -0.63506551]
Best params: {'alpha': 0.034339915870309524, 'l1_ratio': 0.7688000396844558}
Post-tune CV scores: [-0.60329539 -0.60547727 -0.61235485 -0.60602077 -0.62584919]


ElasticNet(alpha=0.034339915870309524, l1_ratio=0.7688000396844558,
           max_iter=500, random_state=3927, selection='random')

## DecisionTree

In [77]:
from sklearn.tree import DecisionTreeRegressor

In [78]:
model = DecisionTreeRegressor(random_state=3927)
params = {"max_depth" : [None, 1, 3, 5, 7, 9],
          "min_samples_leaf": [1, 3, 5, 7, 9],
          "min_weight_fraction_leaf": [0.1, 0.2, 0.3, 0.4, 0.5],
          "max_leaf_nodes": [None, 10, 30, 50, 70, 90]}

model = train(model, X_train, y_train, tune=True, params=params, n_iter=20)
model

Pre-tune CV scores: [-0.66235402 -0.66574107 -0.65846784 -0.66679303 -0.66972977]
Best params: {'min_weight_fraction_leaf': 0.1, 'min_samples_leaf': 3, 'max_leaf_nodes': 70, 'max_depth': None}
Post-tune CV scores: [-0.59148096 -0.59375481 -0.60045088 -0.59816698 -0.61848356]


DecisionTreeRegressor(max_leaf_nodes=70, min_samples_leaf=3,
                      min_weight_fraction_leaf=0.1, random_state=3927)

## SGD

In [80]:
from sklearn.linear_model import SGDRegressor
from scipy.stats import loguniform, uniform

In [81]:
model = SGDRegressor(max_iter=5000, penalty='l2', random_state=3927)
params = {'alpha': uniform(0.001, 1000)}

model = train(model, X_train, y_train, tune=True, params=params, n_iter=10)
model

Pre-tune CV scores: [-0.60184232 -0.60206038 -0.60952616 -0.60570489 -0.62559363]
Best params: {'alpha': 75.64173072984201}
Post-tune CV scores: [-0.62324302 -0.61909415 -0.62862468 -0.6263908  -0.63351724]


SGDRegressor(alpha=75.64173072984201, max_iter=5000, random_state=3927)

In [82]:
model = SGDRegressor(max_iter=5000, penalty='elasticnet', random_state=3927)
params = {'alpha': loguniform(0.001, 1000), 'l1_ratio': uniform(0.1, 0.8)}

model = train(model, X_train, y_train, tune=True, params=params, n_iter=10)
model

Pre-tune CV scores: [-0.6018405  -0.60205898 -0.60952005 -0.60570345 -0.62559216]
Best params: {'alpha': 0.005012444501918985, 'l1_ratio': 0.2919813353920865}
Post-tune CV scores: [-0.60308389 -0.60236847 -0.60717254 -0.60343441 -0.62382585]


SGDRegressor(alpha=0.005012444501918985, l1_ratio=0.2919813353920865,
             max_iter=5000, penalty='elasticnet', random_state=3927)

In [83]:
model = SGDRegressor(max_iter=5000, penalty='elasticnet', random_state=3927)
params = {'alpha': uniform(0.0001, 1), 'l1_ratio': uniform(0.1, 0.8)}

model = train(model, X_train, y_train, tune=True, params=params, n_iter=10)
model

Pre-tune CV scores: [-0.6018405  -0.60205898 -0.60952005 -0.60570345 -0.62559216]
Best params: {'alpha': 0.1340498067109922, 'l1_ratio': 0.7688000396844558}
Post-tune CV scores: [-0.60382075 -0.60341686 -0.61697702 -0.60296978 -0.62210577]


SGDRegressor(alpha=0.1340498067109922, l1_ratio=0.7688000396844558,
             max_iter=5000, penalty='elasticnet', random_state=3927)

In [38]:
model = SGDRegressor(max_iter=5000, penalty='elasticnet', random_state=3927)
params = {'alpha': uniform(0.005, 0.05), 'l1_ratio': uniform(0.1, 0.8)}

model = train(model, X_train, y_train, tune=True, params=params, n_iter=10)
model

Pre-train CV scores: [-0.52058201 -0.80294035 -0.52343453 -0.67453236 -0.66563684]
Best params: {'alpha': 0.01169749033554961, 'l1_ratio': 0.7688000396844558}
Post-train CV scores: [-0.51815211 -0.53837724 -0.53557793 -0.54162234 -0.56373763]


SGDRegressor(alpha=0.01169749033554961, l1_ratio=0.7688000396844558,
             max_iter=5000, penalty='elasticnet', random_state=3927)

## Stacking

In [84]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import StackingRegressor

In [86]:
estimators = [
    ('RIDGE', Ridge(alpha=1, random_state=3927)),
    ('ELNET', ElasticNet(max_iter=500, alpha=0.01, l1_ratio=0.6, selection='random', random_state=3927)),
    ('DTREE', DecisionTreeRegressor(max_leaf_nodes=70, min_samples_leaf=3,
                                    min_weight_fraction_leaf=0.1, random_state=3927)),
    ('SGDR', SGDRegressor(max_iter=5000, penalty='elasticnet', alpha=0.01, l1_ratio=0.6, random_state=3927))
]
model = StackingRegressor(estimators, n_jobs=-1)

model = train(model, X_train, y_train)
model

CV scores: [-27.28249445 -28.71580287 -15.66973739 -22.25587526 -29.54273279]


StackingRegressor(estimators=[('RIDGE', Ridge(alpha=1, random_state=3927)),
                              ('ELNET',
                               ElasticNet(alpha=0.01, l1_ratio=0.6,
                                          max_iter=500, random_state=3927,
                                          selection='random')),
                              ('DTREE',
                               DecisionTreeRegressor(max_leaf_nodes=70,
                                                     min_samples_leaf=3,
                                                     min_weight_fraction_leaf=0.1,
                                                     random_state=3927)),
                              ('SGDR',
                               SGDRegressor(alpha=0.01, l1_ratio=0.6,
                                            max_iter=5000, penalty='elasticnet',
                                            random_state=3927))],
                  n_jobs=-1)

In [87]:
estimators = [
    ('RIDGE', Ridge(alpha=1, random_state=3927)),
    ('ELNET', ElasticNet(max_iter=500, alpha=0.01, l1_ratio=0.6, selection='random', random_state=3927)),
    ('SGDR', SGDRegressor(max_iter=5000, penalty='elasticnet', alpha=0.01, l1_ratio=0.6, random_state=3927))
]
final = DecisionTreeRegressor(max_leaf_nodes=70, min_samples_leaf=3,
                                    min_weight_fraction_leaf=0.1, random_state=3927)
model = StackingRegressor(estimators, final_estimator=final, n_jobs=-1)

model = train(model, X_train, y_train)
model

CV scores: [-0.59551544 -0.5980722  -0.60405917 -0.60086041 -0.61951078]


StackingRegressor(estimators=[('RIDGE', Ridge(alpha=1, random_state=3927)),
                              ('ELNET',
                               ElasticNet(alpha=0.01, l1_ratio=0.6,
                                          max_iter=500, random_state=3927,
                                          selection='random')),
                              ('SGDR',
                               SGDRegressor(alpha=0.01, l1_ratio=0.6,
                                            max_iter=5000, penalty='elasticnet',
                                            random_state=3927))],
                  final_estimator=DecisionTreeRegressor(max_leaf_nodes=70,
                                                        min_samples_leaf=3,
                                                        min_weight_fraction_leaf=0.1,
                                                        random_state=3927),
                  n_jobs=-1)

## Bagging

In [88]:
from sklearn.ensemble import BaggingRegressor
from scipy.stats import randint

In [102]:
model = BaggingRegressor(n_estimators=10, n_jobs=-1, random_state=3927)
params = {'max_samples': randint(1, 100), 'max_features': randint(1, 20)}

model = train(model, X_train, y_train, tune=True, params=params, n_iter=30)
model

Pre-tune CV scores: [-0.47788705 -0.48441964 -0.49308377 -0.50106005 -0.50426299]
Best params: {'max_features': 8, 'max_samples': 96}
Post-tune CV scores: [-0.60069901 -0.59479298 -0.60073053 -0.59615137 -0.62399794]


BaggingRegressor(max_features=8, max_samples=96, n_jobs=-1, random_state=3927)

In [110]:
model = BaggingRegressor(n_estimators=100, n_jobs=-1, random_state=3927)

model = train(model, X_train, y_train)
model

CV scores: [-0.46414895 -0.4710153  -0.48348008 -0.48675993 -0.49328828]


BaggingRegressor(n_estimators=100, n_jobs=-1, random_state=3927)

In [90]:
base = Ridge(alpha=1, random_state=3927)
model = BaggingRegressor(base_estimator=base, n_estimators=20, n_jobs=-1, random_state=3927)
params = {'max_samples': randint(1, 100), 'max_features': randint(1, 20)}

model = train(model, X_train, y_train, tune=True, params=params, n_iter=10)
model

Pre-tune CV scores: [-0.60038772 -0.60371702 -0.60781098 -0.6049524  -0.62425982]
Best params: {'max_features': 17, 'max_samples': 30}
Post-tune CV scores: [-0.61641832 -0.62776915 -0.63349879 -0.62867303 -0.63003017]


BaggingRegressor(base_estimator=Ridge(alpha=1, random_state=3927),
                 max_features=17, max_samples=30, n_estimators=20, n_jobs=-1,
                 random_state=3927)

In [91]:
base = SGDRegressor(max_iter=5000, alpha=0.01, l1_ratio=0.6, penalty='elasticnet', random_state=3927)
model = BaggingRegressor(base_estimator=base, n_estimators=2, n_jobs=-1, random_state=3927)
params = {'max_samples': randint(1, 100), 'max_features': randint(1, 20)}

model = train(model, X_train, y_train, tune=True, params=params, n_iter=10)
model

Pre-tune CV scores: [-0.60772179 -0.60589377 -0.60805996 -0.60706383 -0.62722213]
Best params: {'max_features': 12, 'max_samples': 60}
Post-tune CV scores: [-0.61628121 -0.60219481 -0.61696946 -0.62098255 -0.62340703]


BaggingRegressor(base_estimator=SGDRegressor(alpha=0.01, l1_ratio=0.6,
                                             max_iter=5000,
                                             penalty='elasticnet',
                                             random_state=3927),
                 max_features=12, max_samples=60, n_estimators=2, n_jobs=-1,
                 random_state=3927)

## Boosting

### AdaBoost

In [92]:
from sklearn.ensemble import AdaBoostRegressor

In [93]:
model = AdaBoostRegressor(n_estimators=10, loss='square', random_state=3927)

model = train(model, X_train, y_train)
model

CV scores: [-0.62612785 -0.63610368 -0.70419332 -0.68575288 -0.65371367]


AdaBoostRegressor(loss='square', n_estimators=10, random_state=3927)

In [96]:
base = Ridge(alpha=1, random_state=3927)
model = AdaBoostRegressor(base_estimator=base, n_estimators=2, loss='square', random_state=3927)

model = train(model, X_train, y_train)
model

CV scores: [-0.59987581 -0.60262106 -0.60742434 -0.6040483  -0.62469991]


AdaBoostRegressor(base_estimator=Ridge(alpha=1, random_state=3927),
                  loss='square', n_estimators=2, random_state=3927)

In [97]:
base = SGDRegressor(max_iter=5000, alpha=0.01, l1_ratio=0.6, penalty='elasticnet', random_state=3927)
model = AdaBoostRegressor(base_estimator=base, n_estimators=2, loss='square', random_state=3927)

model = train(model, X_train, y_train)
model

CV scores: [-0.5987626  -0.60642201 -0.61348093 -0.60559309 -0.63237204]


AdaBoostRegressor(base_estimator=SGDRegressor(alpha=0.01, l1_ratio=0.6,
                                              max_iter=5000,
                                              penalty='elasticnet',
                                              random_state=3927),
                  loss='square', n_estimators=2, random_state=3927)

### GradientBoosting

In [98]:
from sklearn.ensemble import GradientBoostingRegressor

In [99]:
model = GradientBoostingRegressor(n_estimators=10, random_state=3927)

model = train(model, X_train, y_train)
model

CV scores: [-0.58158919 -0.57946082 -0.5900043  -0.58762625 -0.6051725 ]


GradientBoostingRegressor(n_estimators=10, random_state=3927)

In [100]:
model = GradientBoostingRegressor(n_estimators=100, random_state=3927)

model = train(model, X_train, y_train)
model

CV scores: [-0.53934113 -0.53961423 -0.55229335 -0.55318057 -0.56992807]


GradientBoostingRegressor(random_state=3927)

## Custom

In [75]:
from scipy.optimize import minimize

def loss_function(beta, X, Y):
    return rmsle(Y, X @ beta)

X_train_tr = transform(pd.DataFrame(np.concatenate((X_train, np.ones((X_train.shape[0], 1))), axis=1), columns=list(X_train.columns) + ['intercept']))
beta_init = np.array([1] * X_train_tr.shape[1])
res = minimize(loss_function, beta_init, args=(X_train_tr[:100], y_train[:100]), options={'maxiter': 100, 'disp': True})
print(res.x)

         Current function value: 12.055089
         Iterations: 59
         Function evaluations: 35574
         Gradient evaluations: 121
[-5.89652413e+02 -7.01089037e+02 -9.62186457e+02  3.33754164e+02
  3.42381794e+02  1.30231496e+03  1.79167827e+03  1.43884259e+03
 -7.49728366e+01  5.79214141e+01  5.28944642e+02  1.83410402e+02
  4.22472647e+02 -4.17908865e+03 -4.98516620e+02 -1.61690452e+00
  1.78767081e+03 -1.72826181e+02 -1.18831036e+02  8.27094063e+02
  1.01015129e+01 -1.75484736e+02  5.85650009e+02 -1.86386872e+02
 -2.81608946e+02 -4.88085043e+02  2.36373631e+02 -6.46890542e+02
 -5.44585001e+02 -7.94931163e+02 -1.52572017e+03 -1.24195282e+03
 -6.74089905e+02 -1.37164174e+03 -3.09696457e+02 -3.64522345e+02
  1.55807784e+03  7.06889666e+02 -9.76006292e+02 -8.33134505e+02
 -1.16224576e+03 -1.70815689e+03  2.13133270e+03  2.09622797e+03
  2.16019188e+03 -1.45156673e+02 -1.12336625e+02 -1.79945344e+02
 -3.95278263e+01  1.07097226e+02 -1.88264431e+02  1.96235380e+02
  3.29597084e+02

# Test

In [111]:
df_pred = test(model, X_test)
df_pred.head()

,id,prediction
0,1,6.827153e+06
1,2,4.969212e+06
2,3,3.669460e+06
3,4,7.122908e+06
4,5,3.680231e+07


In [106]:
model.fit(X_train, y_train)

BaggingRegressor(n_jobs=-1, random_state=3927)

In [107]:
df_pred = test(model, X_test)
df_pred.head()

,id,prediction
0,1,6909428.6
1,2,4893639.5
2,3,3537817.5
3,4,7158078.0
4,5,33075748.8


# Validate

In [112]:
df_pred['prediction'][df_pred['prediction'] <= 100000]

Series([], Name: prediction, dtype: float64)

# Export

In [113]:
df_pred.to_csv("output/submission.csv", index=False)